In [1]:
!pip install einops -q
!pip install wget -q
# !pip uninstall timm -q
!pip install timm==0.4.5 -q

In [2]:
!git clone --branch master --single-branch https://github.com/PVTHust/Speech_project_Vin.git

Cloning into 'Speech_project_Vin'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 322 (delta 180), reused 178 (delta 109), pack-reused 34 (from 1)
Receiving objects: 100% (322/322), 2.66 MiB | 11.73 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import numpy as np

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchaudio
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import copy

In [4]:
# !rm -rf /kaggle/working/*

In [5]:
import sys
import os
sys.path.append(os.path.abspath("/kaggle/working/Speech_project_Vin"))

In [6]:
# First, install the required libraries (run this in a cell if not already installed)
# !pip install torch torchvision tqdm scikit-learn pyyaml

# Import the necessary libraries
import torch
import torch.nn as nn
from tqdm import tqdm
import yaml
from torch import optim
from sklearn.metrics import f1_score, classification_report
from torch.utils.data import DataLoader
from model.Ourmodel import AVClassifier  # Import your model
from dataset.CramedDataset import CramedDataset, load_cremad  # Assuming you have a dataset class
from model.fusion_method import ConcatFusion, SumFusion, FiLM, GatedFusion  # Import fusion methods
from model.manet import manet, RecorderMeter
# Load configuration from YAML file
with open('/kaggle/working/Speech_project_Vin/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Training function
def train_epoch(config, epoch, model, device, dataloader, optimizer, scheduler, save_path=None):
    criterion = nn.CrossEntropyLoss()
    model.train()
    print("Start training ... ")
    _loss = 0

    for step, (spec, image, label) in tqdm(enumerate(dataloader), desc=f'Epoch: {epoch}', total=len(dataloader)):
        spec = spec.to(device)
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        a, v, out = model(spec.unsqueeze(1).float(), image.float())
        loss = criterion(out, label)
        loss.backward()

        optimizer.step()
        _loss += loss.item()

    scheduler.step()

    if save_path:
        torch.save(model.state_dict(), save_path)
        print(f"Model weights saved to {save_path}")

    return _loss / len(dataloader)

# Evaluation function
def eval(config, model, device, dataloader, test=False):
    softmax = nn.Softmax(dim=1)
    n_classes = config['num_classes']

    with torch.no_grad():
        model.eval()
        criterion = nn.CrossEntropyLoss()
        _loss = 0
        golds = []
        preds = []

        for step, (spec, image, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
            spec = spec.to(device)
            image = image.to(device)
            label = label.to(device)

            a, v, out = model(spec.unsqueeze(1).float(), image.float())
            loss = criterion(out, label)
            _loss += loss.item()

            y_hat = torch.argmax(softmax(out), dim=-1)
            golds.extend(label.cpu().numpy())
            preds.extend(y_hat.cpu().numpy())

        wf1 = f1_score(golds, preds, average='weighted')

        if test:
            print(classification_report(golds, preds))

    return _loss / len(dataloader), wf1

# # Gọi hàm load_cremad để tải các dataset
# train_dataset, dev_dataset, test_dataset = load_cremad(config)

# # Khởi tạo DataLoader
# train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
# dev_dataloader = DataLoader(dev_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
# test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# # Kiểm tra độ dài của các DataLoader
# print('Train: {}, Dev: {}, Test: {}'.format(len(train_dataloader), len(dev_dataloader), len(test_dataloader)))


# Initialize the model
model = AVClassifier(config)

# Parallelize model across 2 GPUs
model = torch.nn.DataParallel(model, device_ids=[0, 1])

# Move the model to GPU
model.to('cuda')

# Check the number of GPUs being used
print(f"Using {torch.cuda.device_count()} GPUs")


/kaggle/working/Speech_project_Vin/model/AST.py:138: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast()


Train: 189, Dev: 21, Test: 24
---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True


/kaggle/working/Speech_project_Vin/model/AST.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sdi = torch.load('/kaggle/input/weight-asr/audioset_10_10_0.4593.pth', map

frequncey stride=10, time stride=10
number of patches=300


/kaggle/working/Speech_project_Vin/model/Ourmodel.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  visual_sd = torch.load(config['weight_visual'], map_location='cuda')


Using 2 GPUs


In [ ]:
# Gọi hàm load_cremad để tải các dataset
train_dataset, dev_dataset, test_dataset = load_cremad(config)

# Khởi tạo DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# # Kiểm tra độ dài của các DataLoader
# print('Train: {}, Dev: {}, Test: {}'.format(len(train_dataloader), len(dev_dataloader), len(test_dataloader)))
optimizer = optim.SGD(model.parameters(), lr=config['training']['learning_rate'], momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, config['training']['lr_decay_step'], config['training']['lr_decay_ratio'])
device = 'cuda'
# Train the model
num_epochs = config['training']['epoch']
save_path = config['training']['save_path']
for epoch in range(num_epochs):
    train_loss = train_epoch(config, epoch, model, device, train_dataloader, optimizer, scheduler, save_path=save_path)
    print(f"Train Loss for Epoch {epoch}: {train_loss:.4f}")

    val_loss, wf1 = eval(config, model, device, dev_dataloader, test=True)
    print(f"Validation Loss: {val_loss:.4f}, Weighted F1 Score: {wf1:.4f}")

Start training ... 


Epoch: 0:   0%|          | 0/189 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Epoch: 0:  17%|█▋        | 32/189 [00:56<04:13,  1.61s/it]